# RNGD에서 번역 LLM 실행하기 

- 환경 탐지 (GPU, NPU-FuriosaAI RNGD)

- 번역, 모니터링 실행

- 스코어 계산하기 

- 리더보드에 자동 업데이트 

In [1]:
config_path = "./configuration/config.yaml"

In [2]:
from time import time

from src.translation.one_translate import initialize_translation_environment, translate_text, batch_translate_text
from src.common.load_config import load_all_configurations
from src.metrics.evaluation_transition import evaluate_translation
import os
import json

# Load configuration
all_configs = load_all_configurations(config_path)

# Set ENABLE_MONITORING=true in the environment to enable monitoring
os.environ["ENABLE_MONITORING"] = "true" if "power_consumption" in all_configs["active_metrics"] else "false"
    
# Print the loaded configurations
print("Configurations Loaded Successfully:")
print("Active Metrics:", all_configs["active_metrics"])
print("Device Config:", all_configs["device_config"])
print("Model Config:", all_configs["model_config"])
print("Evaluation Settings:", all_configs["evaluation_settings"])

# Look up imtermediate result folder to avoid redundancy (performance leaderboard)
name_config = (
    f'{all_configs["device_config"]["type"]}-'
    f'{all_configs["device_config"]["model"]}_'
    f'{all_configs["model_config"]["name"]}-'
    f'{all_configs["model_config"]["quantization"]}_'
    f'calib-{all_configs["model_config"].get("calibration", "none")}'
)
result_folder = os.path.join(all_configs["evaluation_settings"]["output_dir"], "ko2en")
result_file = os.path.join(result_folder, f"{name_config}.json")

# Ensure result folder exists
os.makedirs(result_folder, exist_ok=True)

print("----------------------- \n")
if os.path.exists(result_file):
    print(f"Result file already exists at: {result_file}. Skipping computation.")
    proceed_calc = "true"
else:
    print(f"Result file does not exist. Proceeding with computation...")
    proceed_calc = "false"


Configurations Loaded Successfully:
Active Metrics: ['BLEU', 'METEOR', 'BERTScore', 'tps']
Device Config: {'type': 'GPU', 'model': 'A5000', 'count': 1}
Model Config: {'name': 'llama3.1', 'quantization': 'Q4_K_M', 'calibration': 'base'}
Evaluation Settings: {'task': 'translation', 'output_dir': '../result/translation'}
----------------------- 

Result file does not exist. Proceeding with computation...


# 벤치마킹 데이터셋 (FLORES-200)

- 한 -> 영, 영 -> 한 

In [3]:
# Load the CSV file
def load_text_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return [line.strip() for line in lines]

# FLORES-200 데이터 위치 
data_dir = "../data/flores/"

data_eng = load_text_file(f"{data_dir}/devtest.eng_Latn")
data_kor = load_text_file(f"{data_dir}/devtest.kor_Hang")

In [4]:
all_configs["model_config"]["name"]

'llama3.1'

In [5]:
# Initialize the translation environment
try:
    initialize_translation_environment(llm_model=all_configs["model_config"]["name"])
except EnvironmentError as e:
    print(f"Failed to initialize translation environment: {e}")
    exit(1)
    
# 한 -> 영 번역 
num_benchmark = 100
start_time = time() # start time
batch_results = batch_translate_text(data_kor[:num_benchmark], source_language="Korean", target_language="English")
batch_results["elapsed_time"] = time() - start_time  # Add elapsed time to the results
result_folder = os.path.join(all_configs["evaluation_settings"]["output_dir"], "ko2en")
result_file = os.path.join(result_folder, f"{name_config}.json")
print('ko2en translation done')

# Save intermediate result 
with open(result_file, "w") as file:
    json.dump(batch_results, file, indent=4)  # Save the batch results in JSON format
    print(f"Results saved to {result_file}.")
    
# 영 -> 한 번역 
start_time = time()
batch_results = batch_translate_text(data_eng[:num_benchmark], source_language="English", target_language="Korean")
batch_results["elapsed_time"] = time() - start_time  # Add elapsed time to the results
result_folder = os.path.join(all_configs["evaluation_settings"]["output_dir"], "en2ko")
result_file = os.path.join(result_folder, f"{name_config}.json")
print('en2ko translation done')

# Save intermediate result 
with open(result_file, "w") as file:
    json.dump(batch_results, file, indent=4)  # Save the batch results in JSON format
    print(f"Results saved to {result_file}.")


2025-01-05 05:45:39,997 - ERROR - Failed to detect NPUs: [Errno 2] No such file or directory: 'furiosa-smi'
2025-01-05 05:45:39,998 - INFO - Translation environment detected: GPU. Using GPU for translations.
2025-01-05 05:45:45,223 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:46,012 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:46,551 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:47,385 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:48,065 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:48,653 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:49,049 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:45:49,685 - INFO - HTTP Request: POS

ko2en translation done
Results saved to ../result/translate/ko2en/GPU-A5000_llama3.1-Q4_K_M_calib-base.json.


2025-01-05 05:46:42,697 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:44,021 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:44,957 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:46,241 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:47,113 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:47,813 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:48,241 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:49,065 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:49,597 - INFO - HTTP Request: POST http://localhost:11434/api/generate "HTTP/1.1 200 OK"
2025-01-05 05:46:50,105 - INFO - HTTP Request:

en2ko translation done
Results saved to ../result/translate/en2ko/GPU-A5000_llama3.1-Q4_K_M_calib-base.json.
